# YOLO modeļa piemācīšana (fine-tuning) uz malārijas datu kopas

Šajā notebook'ā:

- izmantojam `malaria.yaml` no 1. piezīmjdatora,
- piemācām YOLO modeli (`yolov8n.pt`) uz malārijas patogēnu detekcijas uzdevumu,
- novērtējam modeli uz validācijas testiem,
- apskatām vizuālos rezultātus.

In [ ]:
# Ja ultralytics nav instalēts:
# !pip install ultralytics

from ultralytics import YOLO 
from pathlib import Path
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import torch 

YOLO_ROOT = Path("data/malaria_yolo")
IMAGES_VAL = YOLO_ROOT / "images" / "val"
LABELS_VAL = YOLO_ROOT / "labels" / "val"
MALARIA_YAML = YOLO_ROOT / "malaria.yaml"

infected_classes = [
    "trophozoite",
    "ring",
    "schizont",
    "gametocyte",
]

MALARIA_YAML, MALARIA_YAML.read_text()

In [ ]:
# Izmantojam mazāko YOLO modeli (nano), lai treniņš būtu ātrāks
model = YOLO("yolov8n.pt")  # sākam ar COCO svaru inicializāciju

if torch.backends.cuda.is_available():
    device = 'cuda'     # Ja ir nVidia karte ar labu GPU atmiņu
elif torch.backends.mps.is_available():
    device = 'mps'      # Ja ir MacBook ar M-tipa čipu (Apple Silicon procesori)
else:
    device = 'cpu'      # Noklusējuma variants - aprēķini tiks veikti uz procesora.

print('Modeļa apmācībai izmantojam:', device.upper())    

results = model.train(
    data=str(MALARIA_YAML),  # ceļš uz YAML
    epochs=20,               # var samazināt/samazināt atkarībā no iespējām
    imgsz=640,               # attēla izmērs, ko lietos treniņā
    batch=8,                 # atkarīgs no GPU atmiņas (uz CPU var likt mazāk)
    name="yolo_malaria_v1",  # eksperiments
    device=device,           
)

In [ ]:
# Metriku aprēķins modeļa novērtēšanai
metrics = model.val()
metrics

# mAP50 – vidējais precizitātes/atgūšanas rādītājs pie IoU sliekšņa 0.5,
# precision, recall – cik “tīrs” ir modelis (maz FP) un cik “jutīgs” (maz FN).

In [ ]:
# Ultralytics parasti saglabā modeļus mapē runs/detect/yolo_malaria_v1/weights/best.pt.
best_model_path = Path("runs/detect/yolo_malaria_v1/weights/best.pt")
best_model_path.exists(), best_model_path

In [ ]:
best_model = YOLO(str(best_model_path))
# Validācija uz val kopas, izmantojot labāko modeli
metrics = best_model.val(data=str(MALARIA_YAML))

# metrics ir DetMetrics objekts (Ultralytics klase),
# kuram ir results_dict ar galvenajiem rādītājiem.  
results = metrics.results_dict
results

In [ ]:
print("=== Kopsavilkums par modeļa kvalitāti ===")
print(f"Precizitāte (precision): {results['metrics/precision(B)']:.3f}")
print(f"Recall: {results['metrics/recall(B)']:.3f}")
print(f"mAP@0.5: {results['metrics/mAP50(B)']:.3f}")
print(f"mAP@0.5:0.95: {results['metrics/mAP50-95(B)']:.3f}")
print()
print("Interpretācija:")
print("- Precizitāte: cik liela daļa no atrastajiem objektiem patiešām ir pareizi.")
print("- Recall: cik lielu daļu no visiem patiesajiem objektiem modelis atrod.")
print("- mAP@0.5: \"standarta\" objektu detekcijas metriks (IoU>=0.5).")
print("- mAP@0.5:0.95: stingrāks metriks, ņem vērā dažādus IoU sliekšņus.")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

val_dir = Path("runs/detect/yolo_malaria_v1")
pr_curve_path = val_dir / "BoxPR_curve.png"

if pr_curve_path.exists():
    img_pr = Image.open(pr_curve_path)
    plt.figure(figsize=(10, 10))
    plt.imshow(img_pr)
    plt.axis("off")
    plt.title("Precision–Recall līkne (YOLO val rezultāts)")
    plt.show()
else:
    print("PR_curve.png nav atrasts:", pr_curve_path)

In [ ]:
cm_path = val_dir / "confusion_matrix.png"

if cm_path.exists():
    img_cm = Image.open(cm_path)
    plt.figure(figsize=(10, 10))
    plt.imshow(img_cm)
    plt.axis("off")
    plt.title("Confusion matrix (klases pret klasēm)")
    plt.show()
else:
    print("confusion_matrix.png nav atrasts:", cm_path)

In [ ]:
# Ielādējam labāko modeli
best_model = YOLO(str(best_model_path))

In [ ]:
# Izmantojam tās pašas palīgfunkcijas, ko izmantojām 2. notebook'ā, novērtējot oriģinālo YOLO modeli
from typing import List, Tuple

def load_yolo_labels(label_path: Path):
    boxes = []
    if not label_path.exists():
        return boxes
    with open(label_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            parts = line.strip().split()
            cls_id = int(parts[0])
            xc, yc, w, h = map(float, parts[1:])
            boxes.append((cls_id, xc, yc, w, h))
    return boxes


def yolo_to_xyxy(xc, yc, w, h, img_w, img_h):
    cx = xc * img_w
    cy = yc * img_h
    bw = w * img_w
    bh = h * img_h
    x1 = cx - bw / 2
    y1 = cy - bh / 2
    x2 = cx + bw / 2
    y2 = cy + bh / 2
    return x1, y1, x2, y2


def draw_boxes(image: Image.Image, boxes: List[Tuple[float,float,float,float]], color, labels=None):
    img = image.copy()
    draw = ImageDraw.Draw(img)
    for i, (x1, y1, x2, y2) in enumerate(boxes):
        draw.rectangle([x1, y1, x2, y2], outline=color, width=8)
        if labels is not None:
            draw.text((x1, y1), labels[i], fill=color)
    return img

In [ ]:
val_images = sorted(list(IMAGES_VAL.glob("*.jpg")))[:5]  # 5 piemēri

for img_path in val_images:
    print("Attēls:", img_path.name)
    image = Image.open(img_path).convert("RGB")
    w, h = image.size

    # Ground truth (malārijas patogēni)
    label_path = LABELS_VAL / (img_path.stem + ".txt")
    gt_boxes_yolo = load_yolo_labels(label_path)

    gt_xyxy = []
    gt_labels = []
    for (cls_id, xc, yc, bw, bh) in gt_boxes_yolo:
        x1, y1, x2, y2 = yolo_to_xyxy(xc, yc, bw, bh, w, h)
        gt_xyxy.append((x1, y1, x2, y2))
        gt_labels.append(infected_classes[cls_id])

    # Piemācīts YOLO
    results = best_model(img_path)  # noklusēti: imgsz=640
    pred = results[0]

    pred_xyxy = []
    pred_labels = []
    if pred.boxes is not None and len(pred.boxes) > 0:
        # YOLOv8 dod xyxy normalizētos koordinātes (xyxyn) vai pikseļos (xyxy)
        # Šeit izmantojam xyxy pikseļos
        boxes_xyxy = pred.boxes.xyxy.cpu().numpy()
        conf = pred.boxes.conf.cpu().numpy()
        cls_ids = pred.boxes.cls.cpu().numpy().astype(int)

        for (bx1, by1, bx2, by2), c, cid in zip(boxes_xyxy, conf, cls_ids):
            pred_xyxy.append((bx1, by1, bx2, by2))
            pred_labels.append(f"{best_model.names[cid]} {c:.2f}")

    # Zīmējam
    img_gt = draw_boxes(image, gt_xyxy, color="green", labels=gt_labels)
    img_pred = draw_boxes(image, pred_xyxy, color="red", labels=pred_labels)

    fig, axs = plt.subplots(1, 3, figsize=(20, 10))
    axs[0].imshow(image)
    axs[0].set_title("Oriģinālais attēls")
    axs[0].axis("off")

    axs[1].imshow(img_gt)
    axs[1].set_title("Ground truth (zaļš)")
    axs[1].axis("off")

    axs[2].imshow(img_pred)
    axs[2].set_title("Piemācītais YOLO (sarkans)")
    axs[2].axis("off")

    plt.tight_layout()
    plt.show()